<a href="https://colab.research.google.com/github/jeeyean1004/OpenDataWrangling/blob/master/0613_geo_heatmap_ipynb_%EA%B2%BD%EB%82%A8_%EC%9D%BC%EB%8B%A8%EC%8B%9C%EA%B0%81%ED%99%94%EB%8A%94%EC%99%84%EC%84%B1_%EC%A1%B0%EB%A5%98%EA%B3%A0%EC%9E%A5%EB%A7%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
pip install folium geopandas

In [0]:
#curl -O https://www2.census.gov/geo/tiger/GENZ2015/shp/cb_2015_us_cd114_20m.zip 
#curl 실패, 주소오류 ,다음에 하기

SyntaxError: ignored

# Geographic Heatmap in Python with Folium

__Python 3.4+__  

This notebook has a [companion blog post](https://alcidanalytics.com/p/geographic-heatmap-in-python). In summary it's an example of using folium to generate interactive geographic heatmaps. It also shows how to add arbitrary shapefiles and geojson to leaflet maps with geopandas.

<a href="https://alcidanalytics.com/p/geographic-heatmap-in-python"><img src="https://github.com/mikecunha/pygeo_heatmap/blob/master/images/geo_heatmap_banner.png?raw=1"></a>

__The tools__: 
- [folium](https://github.com/python-visualization/folium) a Python library that outputs HTML with leaflet.js maps
- [geopandas](geopandas.org) extends the datatypes used by pandas to allow spatial operations on geometric types

__The data__: 
- shapefiles and congressional boundaries from [census.gov](https://www.census.gov/geo/maps-data/)
- campaign contribution data from [followthemoney.org](http://www.followthemoney.org/our-data/about-our-data/)

## Load Campaign Contribution Data

These data were aggregated from an export from followthemoney.org. How, will be described in a separate notebook and blog post.

In [0]:
import pandas as pd 

for_map = pd.read_csv('./gdrive/My Drive/raw_data/map/결과6.shp.csv', encoding='cp949')# sep='\t'

for_map.columns

Index(['No', 'row ID', 'add', 'result', 'Frequency', 'Expected', 'Deviation',
       'Percent', 'Row Percent', 'Column Percent', 'Total Row Count',
       'Total Column Count', 'Total Count', 'Cell Chi-Square', 'totaladd',
       'lon', 'lat'],
      dtype='object')

In [0]:
for_map.sort_values('Frequency', ascending=False).head()

,No,row ID,add,result,Frequency,Expected,Deviation,Percent,Row Percent,Column Percent,Total Row Count,Total Column Count,Total Count,Cell Chi-Square,totaladd,lon,lat
266,1347,Row1349,통영시 산양읍,조류접촉,45,24.373131,20.626869,0.457643,48.387097,1.746217,93,2577,9833,17.456424,통영시 산양읍 연화리324,128.397814,34.779091
78,405,Row404,마산시 진전면,조류접촉,37,15.462524,21.537476,0.376284,62.711864,1.435778,59,2577,9833,29.999168,창원시 마산합포구 진전면 동산리51,128.392440,35.122358
89,447,Row446,밀양시 무안면,조류접촉,36,19.393674,16.606326,0.366114,48.648649,1.396973,74,2577,9833,14.219588,밀양시 무안면 동산리564,128.664566,35.535226
61,310,Row309,마산시 내서읍,조류접촉,34,20.704058,13.295942,0.345774,43.037975,1.319364,79,2577,9833,8.538523,창원시 마산회원구 내서읍 중리632,128.516819,35.250048
8,48,Row47,거제시 사등면,조류접촉,33,15.724601,17.275399,0.335605,55.000000,1.280559,60,2577,9833,18.979141,거제시 사등면 사곡리243,128.579679,34.898819


In [0]:
for_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 17 columns):
No                    321 non-null int64
row ID                321 non-null object
add                   321 non-null object
result                321 non-null object
Frequency             321 non-null int64
Expected              321 non-null float64
Deviation             321 non-null float64
Percent               321 non-null float64
Row Percent           321 non-null float64
Column Percent        321 non-null float64
Total Row Count       321 non-null int64
Total Column Count    321 non-null int64
Total Count           321 non-null int64
Cell Chi-Square       321 non-null float64
totaladd              321 non-null object
lon                   321 non-null float64
lat                   321 non-null float64
dtypes: float64(8), int64(5), object(4)
memory usage: 42.7+ KB


## Load 23rd Congr. District Shapefile

In [0]:
!pip install pyshp

     |████████████████████████████████| 225kB 3.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/a6/0c/de/321b5192ad416b328975a2f0385f72c64db4656501eba7cc1a
Successfully built pyshp


In [0]:
import shapefile
from json import dumps

# read the shapefile
shp_path_node='./gdrive/My Drive/raw_data/map/korea_map/encore/wgs84.shp' #encoding='cp949'
reader = shapefile.Reader(shp_path_node)
reader.encoding='cp949' #한글로 하기위해 인코딩
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature", geometry=geom, properties=atr)) 

    # write the GeoJSON file
from json import dumps
geojson = open("pyshp-demo.json", "w")
geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
geojson.close()  # 한덩어리를 buffer로 뭉쳐서 for문을 돌려라가 jason형식임

In [0]:
!pip install geopandas
import geopandas as gpd
import folium

#congr_districts = gpd.read_file('pyshp-demo.json')
#congr_districts = gpd.read_file('zip://gdrive/My Drive/raw_data/map/korea_map/heatmap/SIG_201902.zip')
congr_districts = gpd.read_file('zip://gdrive/My Drive/raw_data/map/pygeo_heatmap-master/data/cb_2015_us_cd114_20m.zip')
#congr_districts = gpd.read_file('zip://'+'data/cb_2015_us_cd114_20m.zip') 원문
# Set datum and projection info for census.gov 2015 Tiger data


#WGS84 위도 / 경도 : 또는
#"+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs""+init=epsg:4326"

#congr_districts.crs = {'datum': 'WGS84', 'ellps': 'WGS84', 'proj':'longlat', 'no_defs':""} #공간조인과 투영을 위해
congr_districts.crs = {'datum': 'NAD83', 'ellps': 'GRS80', 'proj':'longlat', 'no_defs':True}

# Filter out all but the district of interest
district23 = congr_districts[ congr_districts.GEOID == '3623' ]
#district23 = congr_districts[ congr_districts.GEOID == '3623' ]  # 36 = NY, 23 = District 35.238572,128.692463 
#NY 관련해서 없애고 싶은데 못없애고 있음.

     |████████████████████████████████| 901kB 3.4MB/s 
     |████████████████████████████████| 17.9MB 50.3MB/s 
     |████████████████████████████████| 11.2MB 33.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


Initialize a folium map and see what the default crs data is in case we need to convert the shapefile we loaded

In [0]:
distric_map = folium.Map(location=[35.238572,128.692463], zoom_start=7, tiles='cartodbpositron' )
print('default map crs: ',distric_map.crs) #아마 여기서 기본 맵을 바꿀수 있을것 같은데 맵 배경화면을 어떻게 바꿀지..

default map crs:  EPSG3857


In [0]:
# convert it to the projection of our folium openstreetmap
district23 = district23.to_crs({'init':'epsg:3857'})  

Try plotting the congressional district boundary.

(Leaflet.js maps don't render in github notebook previews, [check out the blog post](https://alcidanalytics.com/p/geographic-heatmap-in-python) if you want to see the live interactive maps without having to run this notebook)

In [0]:
folium.GeoJson(district23).add_to(distric_map)
distric_map

## Heatmap

In [0]:
import folium
from folium.plugins import HeatMap

max_amount = float(for_map['Frequency'].max())

hmap = folium.Map(location=[35.238572,128.692463], zoom_start=10, )
#hmap = folium.Map(location=[41.0, -75.5], zoom_start=7, )

#hm_wide = HeatMap( list(zip(for_map."위도".values, for_map."경도".values, for_map.Frequency.values)),
hm_wide = HeatMap( list(zip(for_map.lat.values, for_map.lon.values, for_map.Frequency.values)), 
                   min_opacity=0.2,
                   max_val=max_amount,
                   radius=17, blur=15, 
                   max_zoom=1, 
                 )

folium.GeoJson(district23).add_to(hmap)
hmap.add_child(hm_wide)

In [0]:
import os
hmap.save(os.path.join('./gdrive/My Drive/result', 'shortblackout_bird2.html'))

# Saving png's of Folium Maps

https://github.com/python-visualization/folium/issues/35

These aren't tested, just pasted here as a starting point should someone need them.


PNG로도 만들고 싶은데 못만들어서 포기했습니다.

### Using cutycapt

In [0]:
import os
import subprocess
outdir = "zip://gdrive/My Drive/raw_data/map/pygeo_heatmap-master/results" # this directory has to exist..
#hmap.save("tmp.html")  # heatmap saved above
url = "file://{}/results/geo_heatmap1.html".format(os.getcwd())
outfn = os.path.join(outdir,"geo_heatmap1.png")

subprocess.check_call(["cutycapt","--url={}".format(url), "--out={}".format(outfn)])

FileNotFoundError: ignored

### Using selenium

In [0]:
pip webdriver

ERROR: unknown command "webdriver"


In [0]:
import os
import time
from selenium import webdriver

fn = os.path.join('results', 'geo_heatmap.html')
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
#hmap.save("tmp.html")  # heatmap saved above

browser = webdriver.Firefox()
browser.get(tmpurl)
time.sleep(5)  #Give the map tiles some time to load
browser.save_screenshot('geo_heatmap.png')
browser.quit()

ModuleNotFoundError: ignored